# f) Ngonye Falls Power and Energy Periods



## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|



## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|



## Libraries and Setup

In [20]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [21]:
input_data='./input_data/'
output_data='./output_data/'

model_name='Fixed Tailwater'#'Base Case'


In [22]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'


debt_period=18
equity_period=25
discount_rate=1.0 + 0.15

In [23]:
yearly=pd.read_csv(output_data + out_prefix + "_pe_yearly.csv")

In [24]:
yearly

,WaterYear,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor,Prop4Units,Prop3Units,Prop2Units,Prop1Units,Prop0Units,Days0Units
0,1924,3456.928400,111.133124,0.872827,0.052937,19.905938,18.063098,970,970,178.967823,0.000000,818532.445021,0.519110,0.408219,0.035616,0.109589,0.419178,0.027397,10
1,1925,4504.008836,157.205464,0.566039,0.040630,19.835534,18.063098,970,970,180.597335,17.490911,781338.338915,0.495522,0.317808,0.076712,0.189041,0.416438,0.000000,0
2,1926,3305.305419,192.534369,0.496243,0.065652,19.843020,18.063098,970,970,180.174273,16.193038,837482.623319,0.531128,0.378082,0.060274,0.178082,0.383562,0.000000,0
3,1927,2256.674509,178.800013,0.473067,0.065583,19.831651,18.063098,970,970,178.969625,17.586314,780273.693817,0.494846,0.304110,0.106849,0.178082,0.413699,0.000000,0
4,1928,1587.035405,195.867340,0.502447,0.093256,19.821054,18.063098,970,970,180.298818,18.096686,648470.223008,0.411257,0.172603,0.145205,0.210959,0.471233,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2012,3617.058785,224.058948,0.505204,0.055725,19.806759,18.063098,970,970,179.225616,19.692622,893899.856611,0.566908,0.413699,0.076712,0.128767,0.380822,0.000000,0
89,2013,3578.198961,194.855402,0.497805,0.056469,19.844088,18.063098,970,970,179.469093,16.093068,961928.124811,0.610051,0.446575,0.131507,0.128767,0.293151,0.000000,0
90,2014,1460.437582,205.608866,0.542968,0.100587,19.811896,18.107945,970,970,180.420580,19.445875,789721.469875,0.500838,0.306849,0.117808,0.161644,0.413699,0.000000,0
91,2015,2973.367833,173.346774,0.501598,0.067263,19.842068,18.063098,970,970,179.243671,16.501114,784967.364699,0.497823,0.326027,0.068493,0.169863,0.438356,0.000000,0


In [25]:
sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims=sims.set_index('StartYear')
sims

,DebtEnd,EquityEnd
StartYear,,
1924,1941,1948
1925,1942,1949
1926,1943,1950
1927,1944,1951
1928,1945,1952
...,...,...
1988,2005,2012
1989,2006,2013
1990,2007,2014


In [26]:


o=pd.DataFrame()
for row in sims.itertuples(): 
    f=pd.DataFrame(yearly.loc[(yearly['WaterYear']>=row.Index) & (yearly['WaterYear']<=row.EquityEnd)]['Energy'])
    f['Energy']=f['Energy']/1000
    f=f.reset_index()
    f['YearNo']=f.index+1
    f['StartYear']=row.Index
    f=f.set_index('StartYear').drop('index',axis=1)
    #display(f)
    
    o=o.append(f)

sims=sims.join(o)
#sims=sims.reset_index()
#sims=sims.set_index(['StartYear','YearNo'])
sims
#o

,DebtEnd,EquityEnd,Energy,YearNo
StartYear,,,,
1924,1941,1948,818.532445,1
1924,1941,1948,781.338339,2
1924,1941,1948,837.482623,3
1924,1941,1948,780.273694,4
1924,1941,1948,648.470223,5
...,...,...,...,...
1992,2009,2016,893.899857,21
1992,2009,2016,961.928125,22
1992,2009,2016,789.721470,23


In [27]:
sims['CumEnergy']=sims['Energy'].groupby('StartYear').cumsum()

In [28]:
sims['DiscountedEnergy']=sims['Energy']*(1/(discount_rate**(sims['YearNo']-1)))

In [29]:
sims['CumDiscountedEnergy']=sims['DiscountedEnergy'].groupby('StartYear').cumsum()

In [30]:
sims

,DebtEnd,EquityEnd,Energy,YearNo,CumEnergy,DiscountedEnergy,CumDiscountedEnergy
StartYear,,,,,,,
1924,1941,1948,818.532445,1,818.532445,818.532445,818.532445
1924,1941,1948,781.338339,2,1599.870784,679.424643,1497.957088
1924,1941,1948,837.482623,3,2437.353407,633.257182,2131.214270
1924,1941,1948,780.273694,4,3217.627101,513.042619,2644.256889
1924,1941,1948,648.470223,5,3866.097324,370.764955,3015.021844
...,...,...,...,...,...,...,...
1992,2009,2016,893.899857,21,16641.929195,54.617531,5086.712935
1992,2009,2016,961.928125,22,17603.857319,51.107893,5137.820828
1992,2009,2016,789.721470,23,18393.578789,36.485597,5174.306425


In [31]:
sims_summary=(sims.loc[sims['YearNo']==debt_period]).drop(['Energy','YearNo','DiscountedEnergy','DebtEnd','EquityEnd'],axis=1).rename(columns={'CumEnergy':'DebtEnergy','CumDiscountedEnergy':'DebtDiscountedEnergy'})

sims_summary=sims_summary.join((sims.loc[sims['YearNo']==equity_period])
            .drop(['Energy','YearNo','DiscountedEnergy'],axis=1)
            .rename(columns={'CumEnergy':'EquityEnergy','CumDiscountedEnergy':'EquityDiscountedEnergy'}))
sims_summary.head(3)

,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy
StartYear,,,,,,
1924,14225.590253,5549.848271,1941,1948,19630.521236,5849.747902
1925,14130.763079,5508.264154,1942,1949,19635.245067,5814.657643
1926,14164.914486,5511.744795,1943,1950,19721.964224,5818.642166


In [32]:
sims_summary['DebtAvEnergy']=sims.loc[sims['YearNo']<=debt_period]['Energy'].groupby('StartYear').mean()

In [33]:
sims_summary['EquityAvEnergy']=sims.loc[sims['YearNo']<=equity_period]['Energy'].groupby('StartYear').mean()

In [34]:
sims_summary

,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy
StartYear,,,,,,,,
1924,14225.590253,5549.848271,1941,1948,19630.521236,5849.747902,790.310570,785.220849
1925,14130.763079,5508.264154,1942,1949,19635.245067,5814.657643,785.042393,785.409803
1926,14164.914486,5511.744795,1943,1950,19721.964224,5818.642166,786.939694,788.878569
1927,14109.377625,5448.064501,1944,1951,19815.381956,5760.853810,783.854313,792.615278
1928,14106.552611,5440.204536,1945,1952,19932.954511,5759.032748,783.697367,797.318180
...,...,...,...,...,...,...,...,...
1988,12952.894480,4919.025225,2005,2012,19469.218833,5278.658399,719.605249,778.768753
1989,12972.643614,4720.507668,2006,2013,19543.596563,5083.378474,720.702423,781.743863
1990,13265.808137,4783.575473,2007,2014,19697.363647,5142.126054,736.989341,787.894546


In [35]:
sims_summary.to_csv(output_data + out_prefix + "_pe_sims_summary.csv")

In [36]:
sims.to_csv(output_data + out_prefix + "_pe_sims.csv")